In [ ]:
!pip install flask pyngrok sentence-transformers faiss-cpu pyyaml transformers accelerate ngrok bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login
#login("")


In [ ]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
#Create a new ngrok folder
!mkdir hello-ngrok && cd hello-ngrok

mkdir: cannot create directory ‘hello-ngrok’: File exists


In [ ]:
# =========================================
# Flask + ngrok + RAG API
# =========================================
import threading
from flask import Flask, request, jsonify
from pyngrok import ngrok

# (make sure you already ran cells that define: embedder, retrieve_docs, generate_answer, chat)


# ---- Flask app ----
app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Shoplite RAG API!"

@app.route("/health")
def health():
    return jsonify({"status": "ok"})

@app.route("/ping", methods=["POST"])
def ping():
    data = request.json
    query = data.get("query", "")
    # Direct LLM call (no retrieval)
    answer = chat(query, max_new_tokens=200)
    return jsonify({"answer": answer})

@app.route("/chat", methods=["POST"])
def chat_api():
    data = request.json
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field"}), 400

    # 1) Retrieve docs
    docs = retrieve_docs(query, k=3)

    # 2) Generate RAG answer
    result = generate_answer(query, docs)

    return jsonify(result)


# ---- Run Flask in background ----
def run_flask():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_flask, daemon=True)
thread.start()


# ---- ngrok setup (ask for token at runtime for security) ----
ngrok_token = input("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)

public_url = ngrok.connect(5000, bind_tls=True).public_url
print(" ngrok tunnel established at:", public_url)
print(" Health check:", public_url + "/health")
print(" Ping endpoint:", public_url + "/ping")
print(" Chat endpoint:", public_url + "/chat")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


 ngrok tunnel established at: https://interavailable-villous-jolie.ngrok-free.dev
 Chat endpoint: https://interavailable-villous-jolie.ngrok-free.dev/chat


In [ ]:
# # =========================================
# # Embedding model + Tiny LLM (ungated, fast)
# # =========================================
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# # Load sentence embedder
# embedder = SentenceTransformer("all-MiniLM-L6-v2")  # light + fast

# # Load tiny ungated model (Flan-T5)
# model_id = "google/flan-t5-base"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# pipe = pipeline(
#     "text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=200
# )

# print(" Sentence embedder loaded")
# print(" Tiny Flan-T5 model loaded for quick tests")


In [ ]:
# --- Nuke old bitsandbytes and install fresh, plus Accelerate ---
!pip uninstall -y bitsandbytes -q
!pip install --no-cache-dir -q bitsandbytes accelerate

# --- Show versions + confirm GPU ---
import sys, subprocess, torch, pkgutil, importlib
print("Python:", sys.version)
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())

# Some Colab images preload an old bnb; ensure we import the new one after restart
import pkg_resources
try:
    import bitsandbytes as bnb
    print("bitsandbytes (pre-restart):", bnb.__version__)
except Exception as e:
    print("bitsandbytes import before restart failed (expected sometimes):", repr(e))

# Optional: show GPU
!nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 354.0 MB/s eta 0:00:00
Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Torch: 2.8.0+cu126 | CUDA available: True


/tmp/ipython-input-4075836343.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


bitsandbytes (pre-restart): 0.48.0
Tue Sep 30 22:18:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   78C    P0             34W /   70W |     792MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------------

In [ ]:
!pip install -U autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autoawq: filename=autoawq-0.2.9-py3-none-any.whl size=115106 sha256=b01606ab9ddffe7548d93e33d61da694a6a4621042d5bf4a8b390b95e6a41026
  Stored in directory: /root/.cache/pip/wheels/45/1a/7b/7314b3a958454e8ce349f600829a3f0a6a05aeebf987be1e16
Successfully built autoawq


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch

# Sentence embedder
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Quantized AWQ model
model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Load quantized model with AWQ
model = AutoAWQForCausalLM.from_quantized(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

print(" Sentence embedder loaded")
print(" Llama 3.1 (8B) quantized with AWQ (INT4) loaded")

# === Simple test function ===
def chat(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
print(chat("Q: How do I create a seller account on Shoplite?\nA:"))


/usr/local/lib/python3.12/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!
Replacing layers...: 100%|██████████| 32/32 [00:15<00:00,  2.09it/s]
/usr/local/lib/python3.12/dist-packages/awq/models/base.py:541: UserWarning: Skipping fusing modules because AWQ extension is not installed.No module named 'awq_ext'
  warnings.warn("Skipping fusing modules because AWQ extension is not installed." + msg)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Sentence embedder loaded
 Llama 3.1 (8B) quantized with AWQ (INT4) loaded


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Q: How do I create a seller account on Shoplite?
A: To create a seller account on Shoplite, follow these steps:
1. Go to the Shoplite website and click on the "Sell" tab at the top of the page.
2. Click on "Become a Seller" and fill out the registration form with your basic information.
3. You will receive an email to verify your email address. Click on the link in the email to activate your account.
4. Once your account is activated, you can log in to your seller account and start listing your products.
5. To list a product, click on the "List a Product" button and fill out the product details, including the product name, description, price, and images.
6. You can also connect your existing eBay or Amazon account to Shoplite to import your products and listings.
7. Once you have listed your products, you can manage your inventory, track your sales, and receive payments through Shoplite.
Note: Shoplite is a platform that allows you to sell your products to a global audience, so make su

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# =========================================
# Shoplite Knowledge Base
# =========================================

KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """Shoplite supports two primary account types: Buyer and Seller. Buyers create a free account by providing an email address, choosing a password, and optionally entering a display name. Registration requires email verification; verification links expire after 24 hours. For password security, Shoplite enforces a minimum of 10 characters with at least one numeric and one special character. Users may enable 2FA via authenticator apps in account settings for stronger security. Account recovery uses verified email plus a time-limited token. Sellers must complete an expanded registration flow: business legal name, tax ID, business address, representative contact details, and bank account information for payouts. New seller accounts trigger a verification process: automated KYC checks plus manual review if checks flag inconsistencies. Seller verification typically completes in 2–3 business days; additional documentation may be requested for certain categories (electronics, cosmetics). Shoplite stores basic profile metadata (creation date, account type, last login) visible to the account holder. Account deletion requests are handled via settings and require confirmation emails; deleted accounts may have retention windows for records needed for compliance. The registration system exposes APIs for creating accounts and sending verification emails; rate limits apply per IP to prevent abuse."""
    },
    {
        "id": "doc2",
        "title": "Product Search and Filtering Features",
        "content": """Shoplite search is designed for scale and relevance. The platform supports keyword search, autocomplete (typeahead), semantic search (via embeddings), and advanced filtering. Product records include title, brand, category, price, availability, seller_id, rating, tags, and attributes (size, color). Filters are applied as conjunctive constraints; sorting options include relevance, price (low→high, high→low), newest arrivals, and top-rated. Typeahead uses prefix matching and a cached embedding-based fallback for synonyms and common typos. Search architecture: user query → tokenization → normalized string search → embedding-based re-rank → filter application → paginated results. For better performance, the system maintains per-category inverted indices and precomputed price buckets; category counts are approximate and updated via near-real-time pipelines. The product indexing pipeline accepts bulk updates from sellers and individual SKU updates via API. Search supports fuzzy matches (edit distance threshold), stemming for common languages, and localized price/currency. Admins can set "boost" rules to promote sponsored SKUs or high-margin products; these rules are applied post-retrieval. The search service exposes telemetry for query latency, click-through rates, and zero-result events for iterative tuning."""
    },
    {
        "id": "doc3",
        "title": "Shopping Cart and Checkout Process",
        "content": """The Shoplite cart persists items across sessions for logged-in users and for guests using a temporary cookie. Cart items store SKU id, seller id, quantity, unit price, and applied promotion metadata. The checkout flow is multi-step: address selection, shipping method, payment method, review & confirm. Shipping options are calculated per-seller and aggregated; Shoplite supports combined checkout when multiple sellers are involved, applies separate shipping charges, and computes the final total. Taxes are computed at checkout based on shipping address and product tax classifications; Shoplite integrates with tax-provider APIs for region-specific rules. Promo codes are applied during checkout with validation rules (stackability, expiration, min-spend). Payment methods supported: credit/debit cards (tokenized), direct bank transfer (ACH/SEPA), and third-party wallets (PayPal, Apple Pay). The platform uses PCI-compliant vaulting for card tokens; raw card data never touches Shoplite servers. Order creation occurs after payment authorization; in case of authorization failure the checkout returns a retry option. After successful checkout, an order confirmation email is sent with order ID and expected delivery windows. Abandoned cart flows are supported via scheduled emails if the user is logged in."""
    },
    {
        "id": "doc4",
        "title": "Payment Methods and Security",
        "content": """Shoplite supports multiple payment rails and prioritizes security and compliance. Card payments are processed with a PCI-DSS-certified gateway using tokenization and secure communication (TLS 1.2+). The system supports 3DS for regions where additional authentication is required. Stored payment methods are saved as gateway tokens that can be used for subsequent charges and subscriptions. For refunds, Shoplite implements an idempotent refund API allowing full and partial refunds mapped to original payment methods; merchants can issue refunds if the platform policy and settlement windows allow. For payment disputes/chargebacks, Shoplite collects evidence and helps merchants respond to disputes; contested funds may be placed on hold pending resolution. Sensitive operations are logged with masked values; full logs with PII require elevated access. The platform integrates with fraud detection providers (behavioral scoring, velocity checks) and applies rules (decline or require manual review) for high-risk transactions. For bank transfers and local payment methods, Shoplite provides reconciliation tools in accounting dashboards."""
    },
    {
        "id": "doc5",
        "title": "Order Tracking and Delivery",
        "content": """Once an order is created, Shoplite assigns a tracking workflow. Each shipment is assigned a tracking id and carrier; carriers are selected by sellers or via integrated shipping APIs. Estimated delivery times shown to customers are carrier-provided ETAs adjusted for seller handling times and region calendars. The system records status events (created, packed, picked_up, in_transit, out_for_delivery, delivered, exception) and surfaces them via order details and email/SMS notifications. For multi-seller orders, Shoplite shows per-seller shipment timelines and consolidated tracking when possible. The tracking API supports webhook updates from carriers to update status in near-real-time. Customers can request delivery changes (hold, reschedule) subject to carrier rules and seller acceptance. For lost or delayed shipments, Shoplite provides a claims workflow: customer contacts support → evidence collected (photos, timestamps) → seller/shipper investigation → refund or reship decision. Delivery exceptions escalate to support within 48 hours of detection."""
    },
    {
        "id": "doc6",
        "title": "Return and Refund Policies",
        "content": """Shoplite enforces a platform-level return window of 30 days from delivery for most categories, unless a seller-specific policy extends or restricts that window. Returns require a return authorization (RMA) which is generated when a return is initiated via the order page. Customers must provide reason codes and, for some categories (electronics, fragile), photos of defects. Return shipping labels can be generated by sellers or Shoplite (if the seller opts in); label costs are applied per agreement. Refunds are issued to the original payment method once the returned item is received and inspected or immediately if seller policy allows. For non-returnable items (perishable goods, custom-made products), refunds are handled case-by-case. For fraud prevention, Shoplite flags patterns of excessive returns from the same customer. Merchants can define restocking fees which are applied during refund calculation. The platform tracks return rates per SKU and provides merchants with analytics to reduce return causes."""
    },
    {
        "id": "doc7",
        "title": "Product Reviews and Ratings",
        "content": """Shoplite supports verified purchase reviews and star ratings (1-5). Verified reviews are labeled to indicate purchase verification. The review system allows text, optional photos, and attribute-specific ratings (e.g., fit, build-quality). Reviews are moderated: automated filters remove spam and profanity; flagged items are queued for manual review. Sellers can respond to reviews publicly. Aggregate ratings are computed with Bayesian smoothing to avoid early skew; average rating and total verified reviews are shown on product pages. Shoplite also supports Q&A sections where customers and sellers can answer product questions; answers by verified purchasers are highlighted. Abuse policies prohibit incentivized or fake reviews. The platform supports review reminders triggered after delivery if the customer consents. For legal compliance, Shoplite retains review metadata and stores audit logs of moderation actions."""
    },
    {
        "id": "doc8",
        "title": "Seller Account Setup and Management",
        "content": """Seller onboarding begins with business registration: business legal name, tax id, proof of identity for representative, bank payout info, and business address. Sellers choose store names, upload logos, and set return policies per-store. The seller dashboard includes inventory controls, order management, promotions, payout history, and performance analytics. Sellers can configure shipping methods: fixed-rate, weight-based, or integration with shipping providers. Payout schedule is configurable (daily, weekly) with settlement windows to accommodate chargeback risk. Sellers must adhere to marketplace policies (no prohibited items, correct product descriptions). Violations (intellectual property infringement, counterfeit goods) trigger warnings and possible removal. The platform provides APIs for sellers to update inventory (bulk CSV or API), manage listings (create/update), and receive webhook events for order lifecycle changes. There is also a support center for sellers with tiered SLAs based on program level."""
    },
    {
        "id": "doc9",
        "title": "Inventory Management for Sellers",
        "content": """Inventory management handles stock counts, SKU variants, and reservation during checkout. Sellers can set stock as centralized (global count) or multi-location (per-warehouse). Inventory holds occur at order confirmation to avoid overselling; holds expire if payment isn't completed within configured windows. Sellers can configure pre-order or backorder flags for out-of-stock items, with expected lead times displayed to buyers. The platform supports inventory sync via API or scheduled CSV imports; conflict resolution rules favor the most recent timestamp unless manual override is used. Threshold alerts for low stock can trigger notification emails and auto-reorder suggestions if sellers have supplier integrations. For multi-seller bundles or kits, stock is managed using component availability. Inventory reports provide sell-through rates, days-of-stock, and slow-moving SKU lists to help sellers plan procurement."""
    },
    {
        "id": "doc10",
        "title": "Commission and Fee Structure",
        "content": """Shoplite charges a commission on successful sales; commission percentages vary by category and seller program (standard vs. premium). Fees include: transaction commission, payment processing fee, optional fulfillment fees if Shoplite Fulfillment is used, and optional promotional/sponsored listing fees if sellers buy ad placements. Seller payouts net out commissions and refunds; payout statements provide line-item breakdown for each order. Fee schedule is transparent and available in seller agreements; Shoplite reserves the right to change fees with prior notice. For high-volume sellers, custom fee arrangements can be negotiated. Chargebacks and fraud losses may be deducted from payout reserves until resolved. Taxes on commissions may be applied depending on jurisdiction."""
    },
    {
        "id": "doc11",
        "title": "Customer Support Procedures",
        "content": """Shoplite offers multi-channel support: in-app chat, email, and phone for escalations. Support tiers vary (standard vs. premium). The customer support flow: user opens a ticket → automated triage attempts to resolve using FAQ and RAG-powered assistant → if unresolved, tickets escalate to human agents with priority based on issue severity and SLA. For order issues, support agents have access to order history, shipping events, and communication logs. Agents follow a knowledge base and templated responses to ensure consistent messaging. Escalations to sellers require seller response windows; if the seller doesn't respond, Shoplite may intervene. For fraud, specialized teams handle investigations. Support metrics include first-response time, time-to-resolution, and CSAT. Records of all support interactions are retained for training and audit purposes."""
    },
    {
        "id": "doc12",
        "title": "Mobile App Features",
        "content": """Shoplite mobile apps (iOS and Android) provide a streamlined buying experience with push notifications for order status, promotions, and cart reminders. Key features: biometric login, barcode scanning for price checks, camera-enabled review uploads, and offline browsing for saved catalogs. The app supports express checkout via saved payment tokens and one-tap reorder for recent purchases. Push notification preferences are user-configurable and follow OS guidelines. Mobile-specific UX includes swipeable product carousels and gestures to add items to cart. The app also supports seller notifications for new orders and returns. Performance optimizations include image lazy-loading, low-res placeholders, and caching strategy tuned for intermittent mobile connectivity."""
    },
    {
        "id": "doc13",
        "title": "API Documentation for Developers",
        "content": """Shoplite provides a RESTful API with endpoints for products, orders, customers, inventory, and webhooks. Authentication uses OAuth2 for third-party apps and API keys for server-to-server integrations. Rate limits are tiered by developer program. Endpoints return JSON with consistent schema versioning; breaking changes are communicated through the developer portal. Webhooks cover order lifecycle events, payment events, and inventory updates. SDKs in Python and Node are provided to simplify integration. The API supports pagination, field projection, and filtering; responses include HATEOAS links for navigation. The developer portal has sandbox credentials and sample payloads for testing."""
    },
    {
        "id": "doc14",
        "title": "Security and Privacy Policies",
        "content": """Shoplite is committed to user data protection and privacy. Data handling follows regional privacy laws (GDPR, CCPA) where applicable. PII is minimized and encrypted at rest; access controls enforce least privilege. Security controls: network isolation, application layer WAF, regular penetration testing, and incident response playbooks. Data retention policies vary by data type; transaction and order data are retained for compliance, while marketing preferences are retained per user consent. Shoplite issues privacy notices and allows users to request data exports or deletions subject to legal exceptions. Vendors and partners undergo security assessments to ensure appropriate safeguards."""
    },
    {
        "id": "doc15",
        "title": "Promotional Codes and Discounts",
        "content": """Shoplite supports flexible promo engines for percentage discounts, fixed-amount discounts, free-shipping, and buy-X-get-Y offers. Promotions can be targeted (first-time buyers, cart value thresholds, specific buyer segments), time-limited, or tied to seller campaigns. Coupons can be single-use or multi-use and may be stackable or exclusive based on configuration. Merchant-level controls allow sellers to create and manage promotions subject to platform policies. Promotion eligibility is determined at checkout, and the applied discounts break down line-item to simplify reconciliation. Promo analytics show redemption rates, incremental sales, and campaign ROI. Fraud measures limit promo abuse (velocity checks, per-account limits)."""
    }
]



In [ ]:
# =========================================
# Build FAISS index from knowledge base
# =========================================
texts = [doc["content"] for doc in KNOWLEDGE_BASE]
embeddings = embedder.encode(texts)

import faiss
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Map FAISS index IDs back to docs
id_map = {i: KNOWLEDGE_BASE[i] for i in range(len(KNOWLEDGE_BASE))}

print(" FAISS index built with", index.ntotal, "documents")


 FAISS index built with 15 documents


In [ ]:
# =========================================
# Assistant Prompts (inline YAML) and same as uploaded to github
# =========================================
import yaml

prompt_yaml = """
# RAG Assistant Prompts Configuration
version: "1.0"
created: "2025-09-23"
author: "Your Name"

base_retrieval_prompt:
  role: |
    You are a helpful Shoplite customer service assistant.
  goal: |
    Provide accurate, concise answers using ONLY the provided Shoplite documentation snippets.
  context_guidelines:
    - "Use only information from the provided document snippets."
    - "If the answer requires multiple documents, synthesize clearly and cite each document title in Sources."
    - "If no relevant context is found, refuse with a short explanation and suggest next steps."
  response_format: |
    Answer: [Concise answer based only on context]
    Sources: [List document titles referenced]
    Confidence: [High/Medium/Low]

multi_doc_synth_prompt:
  role: |
    You are a customer support assistant specialized in combining facts from multiple Shoplite documents.
  goal: |
    Synthesize information from multiple retrieved snippets and produce a structured, step-by-step answer.
  context_guidelines:
    - "Show which bullet comes from which document using inline citations (Document Title)."
    - "When policies conflict, prefer platform-level policy documents (Security & Privacy, Returns)."
  response_format: |
    Summary:
    - [Bullet 1 — source: Document Title]
    - [Bullet 2 — source: Document Title]
    Actionable Steps:
    1. [Step — explanation]
    Sources: [Document Title 1, Document Title 2]

refusal_and_clarify_prompt:
  role: |
    You are a conservative Shoplite assistant that avoids hallucination.
  goal: |
    Refuse to answer or ask clarifying questions when the knowledge base lacks evidence.
  context_guidelines:
    - "If retrieved context score < threshold or no documents returned, say you cannot answer with confidence."
    - "Ask for clarifying details (order id, screenshot) when needed."
  response_format: |
    I cannot answer that confidently because [reason].
    Please provide: [needed information].
    Or contact support: [link to help center]

order_status_prompt:
  role: |
    You are an assistant for order-tracking queries; when an order-id is present, use the order-status API snippet only.
  goal: |
    Give the current status, expected delivery, and next recommended action.
  context_guidelines:
    - "Parse order id from user input. Retrieve order-status snippet or API response only."
    - "Redact PII and do not expose internal tokens."
  response_format: |
    Status: [order status]
    ETA: [delivery estimate or 'unknown']
    Next steps: [what the user can do]
    Sources: [Order Tracking and Delivery]
"""

PROMPTS = yaml.safe_load(prompt_yaml)
print(" Prompts loaded:", list(PROMPTS.keys()))


 Prompts loaded: ['version', 'created', 'author', 'base_retrieval_prompt', 'multi_doc_synth_prompt', 'refusal_and_clarify_prompt', 'order_status_prompt']


In [ ]:
# =========================================
# RAG Pipeline Functions
# =========================================

def retrieve_docs(query, k=3):
    """Retrieve top-k documents from FAISS"""
    query_emb = embedder.encode([query])
    D, I = index.search(query_emb, k)
    return [id_map[i] for i in I[0]]

def choose_prompt(query, docs):
    """Pick which prompt config to use"""
    if len(docs) == 0:
        return PROMPTS["refusal_and_clarify_prompt"]
    if "order" in query.lower() and "id" in query.lower():
        return PROMPTS["order_status_prompt"]
    if len(docs) > 1:
        return PROMPTS["multi_doc_synth_prompt"]
    return PROMPTS["base_retrieval_prompt"]


In [ ]:
def generate_answer(question, docs):
    """Format prompt + call LLM"""
    prompt_cfg = choose_prompt(question, docs)

    # Build context + sources
    context = "\n".join([doc["content"] for doc in docs])
    sources = [doc["title"] for doc in docs]

    # Format final prompt
    full_prompt = (
        prompt_cfg["role"] + "\n\n" +
        prompt_cfg["goal"] + "\n\n" +
        prompt_cfg["response_format"] + "\n\n" +
        "Question: " + question + "\n" +
        "Retrieved Context:\n" + context
    )

    # Use your AWQ chat function instead of pipeline
    llm_output = chat(full_prompt, max_new_tokens=300)

    return {"answer": llm_output, "sources": sources}


query = "How do I create a seller account on Shoplite?"
docs = retrieve_docs(query, k=2)
result = generate_answer(query, docs)

print("Q:", query)
print("\nAnswer:\n", result["answer"])
print("\nSources:", result["sources"])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
